In [3]:
import pandas as pd
import numpy as np

nyr = 155
nlon=72
nlat=37
n_month=12
idata=np.empty([nlon,nlat], dtype = int, order = 'C')
gd=np.empty([nlon,nlat,n_month,nyr],order = 'C')
xlat=np.empty(nlat,order = 'C')
xlon=np.empty(nlon,order = 'C')

In [8]:
file = 'hadslp2r.asc'
count = 0
with open(file,"r") as f:
    while True:
        count += 1
        record=[]
        nowRec=[]
        # Get next line from file
        line = f.readline()
        # if line is empty
        # end of file is reached
        if not line:
            break
        
        if count % 38 ==1:
            pass
        else:
            nowRec.append(line)
        if count % 38 ==0:
            record.append(nowRec)
            nowRec=[]

In [ ]:
def write_to_nc_canque(data,file_name_path):
    import netCDF4 as nc
    lonS=np.linspace(119.885,120.536,652)
    latS=np.linspace(29.984,29.358,627)
    da=nc.Dataset(file_name_path,'w',format='NETCDF4')
    da.createDimension('lons',652)  #创建坐标点
    da.createDimension('lats',627)  #创建坐标点
    da.createVariable("lon",'f',("lons"))  #添加coordinates  'f'为数据类型，不可或缺
    da.createVariable("lat",'f',("lats"))  #添加coordinates  'f'为数据类型，不可或缺
    da.variables['lat'][:]=latS     #填充数据
    da.variables['lon'][:]=lonS     #填充数据
    
    da.createVariable('u','f8',('lats','lons')) #创建变量，shape=(627,652)  'f'为数据类型，不可或缺
    da.variables['u'][:]=data       #填充数据
    da.close()
 
write_to_nc_canque(one,'D://new.nc')
'''
输入的data的shape=(627,652)
'''
def write_to_nc_wanmei(data,file_name_path):
    import netCDF4 as nc
    lonS=np.linspace(119.885,120.536,652)
    latS=np.linspace(29.984,29.358,627)
    da=nc.Dataset(file_name_path,'w',format='NETCDF4')
    da.createDimension('lon',652)  #创建坐标点
    da.createDimension('lat',627)  #创建坐标点
    da.createVariable("lon",'f',("lon"))  #添加coordinates  'f'为数据类型，不可或缺
    da.createVariable("lat",'f',("lat"))  #添加coordinates  'f'为数据类型，不可或缺
    da.variables['lat'][:]=latS     #填充数据
    da.variables['lon'][:]=lonS     #填充数据
 
    da.createVariable('u','f8',('lat','lon')) #创建变量，shape=(627,652)  'f'为数据类型，不可或缺
    da.variables['u'][:]=data       #填充数据 
    da.close()
 
write_to_nc_wanmei(one,'D://new1.nc')